In [1]:

import pyNN.nest as sim
from pyNN.utility import get_simulator, normalized_filename

import numpy as np

def get_pop():

    SENSORPARAMS = {'v_rest': -60.5,
                    'cm': 0.025,
                    'tau_m': 10.,
                    'tau_refrac': 10.0,
                    'tau_syn_E': 2.5,
                    'tau_syn_I': 2.5,
                    'e_rev_E': 0.0,
                    'e_rev_I': -75.0,
                    'v_thresh': -60.0,
                    'v_reset': -60.5}

    GO_ON_PARAMS = {'v_rest': -60.5,
                    'cm': 0.025,
                    'tau_m': 10.0,
                    'e_rev_E': 0.0,
                    'e_rev_I': -75.0,
                    'v_reset': -61.6,
                    'v_thresh': -60.51,
                    'tau_refrac': 10.0,
                    'tau_syn_E': 2.5,
                    'tau_syn_I': 2.5}

    population = sim.Population(8, sim.IF_cond_alpha())
    population[0:5].set(**SENSORPARAMS)
    population[5:6].set(**GO_ON_PARAMS)
    population[6:8].set(**SENSORPARAMS)

    syn_params = {'U': 1.0, 'tau_rec': 1.0, 'tau_facil': 1.0}

    # Synaptic weights
    WEIGHT_RED_TO_ACTOR = 1.5e-4
    WEIGHT_RED_TO_GO_ON = 1.2e-3  # or -1.2e-3?
    WEIGHT_GREEN_BLUE_TO_ACTOR = 1.05e-4
    WEIGHT_GO_ON_TO_RIGHT_ACTOR = 1.4e-4
    DELAY = 1

    # Connect neurons
    CIRCUIT = population

    SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_RED_TO_ACTOR),
                                    delay=DELAY, **syn_params)
    sim.Projection(presynaptic_population=CIRCUIT[2:3],
                   postsynaptic_population=CIRCUIT[7:8],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='excitatory')
    sim.Projection(presynaptic_population=CIRCUIT[3:4],
                   postsynaptic_population=CIRCUIT[6:7],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='excitatory')


    SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_RED_TO_GO_ON),
                                    delay=DELAY, **syn_params)
    sim.Projection(presynaptic_population=CIRCUIT[0:2],
                   postsynaptic_population=CIRCUIT[4:5],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='inhibitory')
    sim.Projection(presynaptic_population=CIRCUIT[0:2],
                   postsynaptic_population=CIRCUIT[5:6],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='inhibitory')

    SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_GREEN_BLUE_TO_ACTOR),
                                    delay=DELAY, **syn_params)
    sim.Projection(presynaptic_population=CIRCUIT[4:5],
                   postsynaptic_population=CIRCUIT[7:8],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='excitatory')

    SYN = sim.TsodyksMarkramSynapse(weight=abs(WEIGHT_GO_ON_TO_RIGHT_ACTOR),
                                    delay=DELAY, **syn_params)
    sim.Projection(presynaptic_population=CIRCUIT[5:6],
                   postsynaptic_population=CIRCUIT[7:8],
                   connector=sim.AllToAllConnector(),
                   synapse_type=SYN,
                   receptor_type='excitatory')

    return population    


class SetRate(object):
    """
    A callback which changes the firing rate of a population of poisson
    processes at a fixed interval.
    """

    def __init__(self, source, interval=20.0):
        assert isinstance(source.celltype, sim.SpikeSourcePoisson)
        self.source = source
        self.interval = interval
        self.rate = 0.00001
        
    def __call__(self, t):
        # t: time in ms
        try:
            self.source.set(rate=self.rate)
        except StopIteration:
            pass
        return t + self.interval

class spike_generator(object):
    min_rate = 0.0005
    def __init__(self, population, interval=20.0, count_red_left=1/3, count_red_right=1/3, count_non_red=1/3):
        
        params_source=  {'start': 0.0, 'duration': float("inf"), 'rate': 0.0}
        params_connec= {'weight':0.00015, 'receptor_type':'excitatory','delay':0.1}
        
        self.ssp_red_left_eye = sim.create(sim.SpikeSourcePoisson,  params_source)
        self.ssp_red_right_eye = sim.create(sim.SpikeSourcePoisson,  params_source)
        self.ssp_green_blue_eye = sim.create(sim.SpikeSourcePoisson,  params_source)
        self.interval = interval
        sim.connect(self.ssp_red_left_eye, population[slice(0, 3, 2)], **params_connec)
        sim.connect(self.ssp_red_right_eye, population[slice(1, 4, 2)], **params_connec)
        sim.connect(self.ssp_green_blue_eye, population[4], **params_connec)
        
        self.rate_red_left_eye = SetRate(self.ssp_red_left_eye, self.interval)
        self.rate_red_right_eye = SetRate(self.ssp_red_right_eye, self.interval)
        self.rate_green_blue_eye = SetRate(self.ssp_green_blue_eye, self.interval)
    
    def get_callbacks(self):
        return [self.rate_red_left_eye,self.rate_red_right_eye,self.rate_green_blue_eye]

    def set_rates_from_ratios(self, count_red_left, count_red_right, count_non_red):
        if count_red_left <= self.min_rate:
            count_red_left = self.min_rate
        if count_red_right <= self.min_rate:
            count_red_right = self.min_rate
        if count_non_red <= self.min_rate:
            count_non_red = self.min_rate
        self.rate_red_left_eye.rate = 2*2000.0 * count_red_left
        self.rate_red_right_eye.rate = 2*2000.0 * count_red_right
        self.rate_green_blue_eye.rate = 75.0 * count_non_red

class output_generator(object):
    def __init__(self, population):
        """parameters taken from:
https://developer.humanbrainproject.eu/docs/projects/hbp-nrp-cle/1.3.8/
modules/hbp_nrp_cle/brainsim/pynn/devices/__PyNNLeakyIntegratorTypes.html
"""
        LI_create = {'v_thresh': float('inf'),'cm': 1.0,'tau_m': 10.0,'tau_syn_E': 2.,
            'tau_syn_I': 2.,'v_rest': 0.0,'v_reset': 0.0,'tau_refrac': 0.1,}
        LI_connect = {'weight':0.00015, 'receptor_type':'excitatory','delay':0.1}
        self.left = sim.create(sim.IF_curr_alpha, LI_create)
        self.right = sim.create(sim.IF_curr_alpha, LI_create)
        sim.connect(population[6], self.left, **LI_connect)
        sim.connect(population[7],self.right,  **LI_connect)
        # sim.connect(sg.ssp_red_right_eye,output2,  **LI_connect)

        # sim.connect(output2, population[7], **LI_connect)
        sim.initialize(self.left, v=self.left.get('v_rest'))
        sim.initialize(self.right, v=self.right.get('v_rest'))
        self.left.record('v')
        self.right.record('v')
    
    def get_current_voltage(self):
        return self._get_current_voltage(self.left), self._get_current_voltage(self.right)
    def _get_current_voltage(self, cell):
        return cell.get_v().segments[0].filter(name="v")[0][-1,0]
        

/opt/conda/lib/python3.7/site-packages/pyNN/nest/__init__.py:55: UserWarning: Unable to install NEST extensions. Certain models may not be available.
Further details: DynamicModuleManagementError in Install: Module 'pynn_extensions' could not be opened.
The dynamic loader returned the following error: 'file not found'.

Please check LD_LIBRARY_PATH (OSX: DYLD_LIBRARY_PATH)!
  warnings.warn("Unable to install NEST extensions. Certain models may not be available.\nFurther details: {}".format(err))


In [2]:


sim.setup(timestep=0.1,min_delay=0.1,max_delay=4.0)
population=get_pop()
sim.initialize(population, v=population.get('v_rest'))
sg = spike_generator(population)
og = output_generator(population)
interval = 20.0
population.record('v')
population.record('spikes')

In [ ]:
import run_direct
import pybullet as p
import numpy as np
%matplotlib notebook

physicsClient = p.connect(p.DIRECT)  # p.DIRECT for non-graphical version
world = run_direct.World()
robot = run_direct.Husky(brain=run_direct.Brain())
robot.maxForce = 150
robot.targetVelocity = 20

In [4]:
# lets simulate a closed loop engine

simuluated_ratio = [(0.0,0.0,0.999),
(0.05,0.05,0.9),
(0,0.1,0.9),
(0.1,0.1,0.9),
(0,0.1,0.9),
(0,0.1,0.9),
(0,0.1,0.9),
(0.1,0.,0.9),
(0.2,0.0,0.9),
(0.0,0.0,0.999),
(0.0,0.0,0.999)]

for ratio in simuluated_ratio:
    sg.set_rates_from_ratios(*ratio)
    sim.run(400.0, callbacks=sg.get_callbacks())
    print(og.get_current_voltage())


(array(5.84297312e-08) * mV, array(0.00053929) * mV)
(array(2.48230197e-25) * mV, array(0.00036585) * mV)
(array(0.00066588) * mV, array(0.00012778) * mV)
(array(0.00052806) * mV, array(0.00047515) * mV)
(array(0.00073208) * mV, array(1.90498079e-06) * mV)
(array(0.00050576) * mV, array(8.09303326e-24) * mV)
(array(0.00071548) * mV, array(3.43820723e-41) * mV)
(array(7.68048755e-17) * mV, array(0.00080884) * mV)
(array(1.71671048e-10) * mV, array(0.00062025) * mV)
(array(1.04663487e-12) * mV, array(0.00042545) * mV)
(array(4.44647572e-30) * mV, array(0.00035318) * mV)


In [5]:
%matplotlib widget
import matplotlib.pyplot as plt
from pyNN.utility.plotting import Figure, Panel
from quantities import mV

plt.subplot(3, 1, 1)
for spiketrain in population.get_data().segments[0].spiketrains:
    y = np.ones_like(spiketrain) * spiketrain.annotations['source_id']
    plt.plot(spiketrain, y, '.')
    plt.setp(plt.gca().get_xticklabels(), visible=True)
    
def plot_voltage(pop, with_separtor=False):
    vm = pop.get_data().segments[0].filter(name="v")[0]
    for c in vm.get_channel_index():
        label = "Neuron %d" % vm.annotations['source_ids'][c]
        if with_separtor:
            distancer = c*7
            a = np.array(vm[:,c]) + distancer
            plt.plot(vm.times, np.full(a.shape, -60.5+ distancer), color='#f5f5f5', linestyle='dashed')
        else:
            a = np.array(vm[:,c])
        plt.plot(vm.times, a, label=label)
    plt.ylabel("%s (%s)" % (vm.name, vm.units._dimensionality.string))
    plt.setp(plt.gca().get_xticklabels(), visible=True)
    plt.show()


plt.subplot(3, 1, 2)
plot_voltage(population,True)
plt.subplot(3, 1, 3)
plot_voltage(og.left)
plot_voltage(og.right)
# plt.legend()

FigureCanvasNbAgg()